In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# setup disply parameters
from matplotlib import pylab as plt
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter
float_formatter = StrMethodFormatter('{x:0.03f}')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('figure', figsize=(18, 6)) # set figure size
plt.rc("animation", html="html5")
import random
random.seed(0)
import numpy as np
np.random.seed(0)

In [ ]:
import pandas as pd
import numpy as np
from time import sleep
from pprint import pprint

## Developing an agent for SCML2024 (Standard)

In 2024, we introduced a new implementation of the SCML-Standard track which simplified its API making it exactly the same as the simpler SCML-OneShot track. The older version of the game is not supported anymore.

**SCML-OneShot brief introduction** Please refer to the first tutorial for a brief introduction about the world simulated in this game as well as pointers to more information. We will assume knowledge of at least this brief introduction in the remainder of this tutorial.

In [ ]:
from collections import defaultdict
import random
from negmas import ResponseType, SAOResponse
from scml.std import *
from scml.std.common import is_system_agent
from pprint import pprint

def try_agent(agent_type, n_processes=3, **kwargs):
    """Runs an agent in a world simulation against a randomly behaving agent"""
    return try_agents([GreedyStdAgent, SyncRandomStdAgent, agent_type], n_processes,**kwargs)

def try_agents(agent_types, n_processes=3, n_trials=5, draw=True, agent_params=None, year=2024):
    """
    Runs a simulation with the given agent_types, and n_processes n_trial times.
    Optionally also draws a graph showing what happened
    """
    type_scores = defaultdict(float)
    counts = defaultdict(int)
    agent_scores = dict()
    for _ in range(n_trials):
        p = n_processes if isinstance(n_processes, int) else random.randint(*n_processes)
        cls = {2024: SCML2024StdWorld}[year]
        world = cls(
            **cls.generate(agent_types, agent_params=agent_params, n_steps=10, 
            n_processes=p, random_agent_types=True), construct_graphs=True
        )
        world.run()

        all_scores = world.scores()
        for aid, agent in world.agents.items():
            if is_system_agent(aid):
                continue
            key = aid if n_trials == 1 else f"{aid}@{world.id[:4]}"
            agent_scores[key] = (
                 agent.type_name.split(':')[-1].split('.')[-1],
                 all_scores[aid],
                 '(bankrupt)' if world.is_bankrupt[aid] else ''
                )
        for aid, agent in world.agents.items():
            if is_system_agent(aid):
                continue
            type_ = agent.type_name.split(':')[-1].split('.')[-1]
            type_scores[type_] += all_scores[aid]
            counts[type_] += 1
    type_scores = {k: v/counts[k] if counts[k] else v for k, v in type_scores.items()}
    if draw:
        world.draw(
            what=["contracts-concluded"],
            steps=(0, world.n_steps - 1),
            together=True, ncols=1, figsize=(20, 20)
        )
        plt.show()

    return world, agent_scores, type_scores

def analyze_contracts(world):
    """
    Analyzes the contracts signed in the given world
    """
    import pandas as pd
    data = pd.DataFrame.from_records(world.saved_contracts)
    return data.groupby(["seller_name", "buyer_name"])[["quantity", "unit_price"]].mean()


def print_agent_scores(agent_scores):
    """
    Prints scores of individiual agent instances
    """
    for aid, (type_, score, bankrupt) in agent_scores.items():
        print(f"Agent {aid} of type {type_} has a final score of {score} {bankrupt}")

def print_type_scores(type_scores):
    """Prints scores of agent types"""
    pprint(sorted(tuple(type_scores.items()), key=lambda x: -x[1]))

Now we are ready to develop a do-nothing agent:

In [ ]:
class MyStdDoNothing(StdPolicy):
    """My Agent that does nothing"""
    def act(self, state):
        return { partner: SAOResponse(ResponseType.END_NEGOTIATION, None)
                for partner in state.my_suppliers + state.my_consumers}
    
world, ascores, tscores = try_agent(MyStdDoNothing, n_trials=1)

In all of the graph representing world simulations, we use short names that represent the type of the agent. For example an agent named `01Gr@0` is an agent of type `GreedyStdAgent` at production level 1 that was the third agent to create. `MSD` here is a shorthand for `MyStdDoNothingAgent`.

Looking at the `contracts-concluded`, we can see that none of the concluded contracts involved our do-nothing agent. Nevertheless, these agents still had *exogenous contracts* which means that they will lose money. A do-nothing agent will usually lose money in this game.

Let's check the scores of different agents to confirm:

In [ ]:
print_agent_scores(ascores)

Notice that our do-nothing agent always loses money. That is because it cannot get any contracts from negotiation to satisfy its needs from the exogenous contracts but it still has to pay for disposal cost and shortfall penalty.

In [ ]:
print_type_scores(tscores)

We can also have a look at the *exogenous* contracts that drive the market.

In [ ]:
analyze_contracts(world)

There are few things to note about the distribution of the *exogenous* contracts:

- The unit price of the raw material is always lower than that of the final product. This is the source of profitability in this market.
- Each agent has a different mean and standar deviation for the quantities in its exogenous contracts. This means that different agents will have different utility functions but these utility functions for different steps are related because the exogenous contract is sampled from some common distribution for each agent for all the steps which makes learning more useful in the game.

### Building your own agent

A standard agent needs only to do negotiation. The simplest possible version (`MyDoNothingAgent` above) just responded to offers from its partners and proposed new offers to them.

#### Your AWI

As described in a [previous tutorial](https://scml.readthedocs.io/en/latest/tutorials/01.run_scml2020.html), your agent can sense and act in the simulation by accessing methods and properties of its AWI which is accessible at any time as:

```python
self.awi
```

You can see all of these methods and properties specific for the **StdAWI** and its descendents [here](https://scml.readthedocs.io/en/latest/api/scml.std.StdAWI.html).

#### Your ufun

The Std game has the advantage that it is possible at the end of each simulation step (day) to calculate **exactly** the profit you will be getting for the set of contracts you have (either through negotiation or as exogenous contracts). We provide a utility function class ([StdUtilityFunction](https://scml.readthedocs.io/en/latest/api/scml.std.StdUFun.html) which can be used normally as any NegMAS [UtilityFunction](http://www.yasserm.com/negmas/api/negmas.preferences.UtilityFunction.html). This ufun is available to your all the time (a new one is created for each simulation step) and is accessible as:

```python
self.ufun
```

The most important services this ufun class provides for you are the following:

- `from_offers`: This method receives a list of outcomes and a list of booleans indicating whether each of them is for buying or for selling. It returns to you the profit you will get if all of these outcomes _and nothing else_ became contracts. An outcome is just a tuple (quantity, delivery time, unit price). You can use this callback during negotiation to judge hypothetical agreements with your partners. **Note that only offers to be delivered today will be considered**
- `from_contracts`: This method is the same as `from_offers` but it receives a list of `Contract` objects. It is useful after all negotiations are finished to calculate the profit you will be getting for this step. **Note that only contracts to be delivered today will be considered**
- `is_breach`: will tell you whether or not getting the given total input and output quantities will make you cause a breach. Notice that breaches are expected in the Standard track as any mismatch in the quantities of inputs and outputs will constitute a breach.
- `breach_level`: returns a value between zero and one specifying the level of breach that will be recorded for a given total input and output quantities.
- `find_limit`: finds either the maximum or the minimum possible profit (minimum profit is maximum loss) attainable in the current simulation step (day). This is useful when you want to normalize utility values between zero and one. Two of the agents we will develop during this tutorial will use this feature.
- `max_utility`, `min_utility`: give the maximum and minimum utilities/profits attainable. Note that you must prepare them by calling `find_limit`. We will go into how to do that later.
- `best`, `worst`: give more information about the cases of maximum and minimum profit (i.e. the total input and output quantity needed, the prodcible quantity, best possible prices for buying and selling, etc). Again, these are not available except after calling `find_limit`.

#### Your callbacks

Your agent needs to implement methods that are called by the system at various time during the negotiation. You can find a full list in the [game description](https://yasserfarouk.github.io/files/scml/y2024/scml2024.pdf).

The most important ones are:

- `init()` called once at the beginning of the simulation (i.e. before the first day starts). At this point, your AWI is set but you should not assume anything else.
- `before_step()` called at the **beginning** of _every day_. At this point, your `ufun` is set and market information is available.
- `step()` called at the **end** of _every day_. You can use this to analyze what happened during the day and modify your strategy in the future.
- `on_negotiation_success()`/`on_negotiation_failure()` called after each negotiation is concluded to let you know what happened in it.
- Depending on your base-class, you will also need to implement methods that allow you to control negotiations. These will be explained in details in the following sections but here is a summary:
  - **StdAgent** If your agent is based on `StdAgent`, you will get a `propose()` call when you need to offer something to one of our partners during negotiation and `respond()` when asked to respond to one of its offers.
  - **StdSyncAgent** If your agent is based on `StdSyncAgent` you will get a call to `first_proposals()` once every day to set your first proposal in all negotiations and a `counter_all()` call to counter offers from your partners. The system will try to always give you one offer from each partner in the `counter_all()` call but that is not guaranteed and sometimes it may be called with a subset of the offers.
  - **StdSingleAgreementAgent** If your agent is based on `StdSingleAgreementAgent` you will have to implement `is_acceptable()` to decide if a given offer is acceptable to you, `best_offer()` to find the _best_ offer in a given negotiation for your agent and `is_better()` to compare two offers. Once you implement these, the agent will implement all callback for you trying to get **a single** agreement that maximizes your utility. Note that, again, it is not guaranteed that you will get a single agreement at the end but the system will try its best to achieve that.

Now we can start working on our agent.

There are three base classes for one-shot agents (`StdAgent`, `SyncStdAgent`, and `StdSingleAgreementAgent`). We will discuss them in more details in what follows.

### StdAgent

This is the base class of all agents for SCML-Std. Both `SyncStdAgent` and `SingleAgreementStdAgent` inherit from this class and provide support for a simplified way of developing your agent (or so we think). It is perfectly OK to use `StdAgent` directly as the base of your agent.

We have already seen the `StdAgent` class for which you need to override `propose` and may also override `respond` to handle negotiations independently. The `propose` method receives the negotiation state (an object of the type `SAOState` including among other things the current negotiation step, relative time, last offer, etc) and is required to return an `Outcome` (See `negmas` documentation) as an offer. The `respond` method receives a negotiation state and an offer (`Outcome`) from the opponent and needs to respond to it by a decision from the `ResponseType` enumeration (`REJECT_OFFER`, `ACCEPT_OFFER`, and `END_NEGOTIATION`). Other than these two negotiation related callbacks, the agent receives an `init` call just after it joins the simulatin and a `step` call after each step. The agent is also informed about failure/success of negotiations through the `on_negotiation_success`/`on_negotiation_failure` callbacks. That is all. A one-shot agent needs to only think about what should it do to respond to each of these six callbacks. All of these callbacks except `propose` are optional.

#### Simple StdAgent

We have already seen how to develop a do-nothing agent using the `StdAgent` class. Let's try to develop some more meaningful agent using the same base class.


In [1]:
class SimpleAgent(StdAgent):
    """A greedy agent based on StdAgent"""

    def propose(self, negotiator_id: str, state) -> "Outcome":
        return self.best_offer(negotiator_id)

    def respond(self, negotiator_id, state, source=''):
        offer = state.current_offer
        return (
            ResponseType.ACCEPT_OFFER
            if offer[QUANTITY] <= self._needed(negotiator_id)
            else ResponseType.REJECT_OFFER
        )

    def best_offer(self, negotiator_id):
        ami = self.get_nmi(negotiator_id)
        if not ami:
            return None
        my_needs = self._needed(negotiator_id)
        if my_needs <= 0:
            return self.best_future_offer(ami, negotiator_id)
        quantity_issue = ami.issues[QUANTITY]
        unit_price_issue = ami.issues[UNIT_PRICE]
        offer = [-1] * 3
        offer[QUANTITY] = max(
            min(my_needs, quantity_issue.max_value),
            quantity_issue.min_value
        )
        offer[TIME] = self.awi.current_step
        if self._is_selling(ami):
            offer[UNIT_PRICE] = unit_price_issue.max_value
        else:
            offer[UNIT_PRICE] = unit_price_issue.min_value
        return tuple(offer)
    
    def best_future_offer(self, ami, negotiator_id):
        time_issue = ami.issues[TIME]
        times = list(time_issue.all)
        random.shuffle(times)
        for t in times:
            needed = self._future_needs(negotiator_id, t)
            if needed <= 0:
                continue
            offer = [-1] * 3
            quantity_issue = ami.issues[QUANTITY]
            unit_price_issue = ami.issues[UNIT_PRICE]
            mx = max(
                min(needed, quantity_issue.max_value), quantity_issue.min_value
            )
            # never contract offer more than production capacity
            mx = max(0, min(mx, self.awi.n_lines * (t - self.awi.current_step)))
            if mx < 1:
                continue
            offer[QUANTITY] = random.randint(
                max(1, int(0.5 + mx * self.awi.current_step / self.awi.n_steps)), mx
            )
            offer[TIME] = t
            if self._is_selling(ami):
                offer[UNIT_PRICE] = unit_price_issue.max_value
            else:
                offer[UNIT_PRICE] = unit_price_issue.min_value
            return tuple(offer)
        return None

    
    def is_seller(self, negotiator_id):
        return negotiator_id in self.awi.current_negotiation_details["sell"].keys()

    def _needed(self, negotiator_id=None):
        if self.awi.is_middle_level:
            return self.awi.n_lines
        return self.awi.needed_sales if self.is_seller(negotiator_id) else self.awi.needed_supplies
    
    def _future_needs(self, negotiator_id, t):
        return self.awi.n_lines - sum(
            (
                self.awi.future_sales
                if negotiator_id in self.awi.my_consumers
                else self.awi.future_supplies
            )
            .get(t, dict())
            .values()
        )

    
    def _is_selling(self, ami):
        return ami.annotation["product"] == self.awi.my_output_product
    
world, ascores, tscores = try_agent(SimpleAgent)

NameError: name 'StdAgent' is not defined

Let's see how well did this agent behave:

In [ ]:
print_type_scores(tscores)

This simple agent is better than the random agent. Let's understand how it works:

The main idea of this agent is pretty simple. It tries to *secure* as much of its needs (sales/supplies) as possible in every negotiation at the best possible price for itself.

To achieve this goal, the agent uses the fact that the `AWI` already keeps track of this information as `needed_supplies` and `needed_sales`.
Therefore, it defines a helper that calculates the amount it needs by subtracting the exogenous quantity it has from the amount it secured

```python
def _needed(self):
    self.awi.needed_sales if self.is_seller(negotiator_id) else self.awi.needed_supplies
```

where it uses `needed_sales` if the current negotiation is for selling and `needed_supplies` otherwise. Now that the agent can calculate how much it needs to buy/sell, it implements the negotiation related call-backs (`propose` and `respond`). 

Here is the full implementation of `propose`:
```python
def propose(self, negotiator_id: str, state) -> "Outcome":
        return self.best_offer(negotiator_id)
```

The agent is always offering its best offer which is calculated in the `best_offer` method to be discussed later. It does not conceed at all.

Responding to opponent offers is also simple:

- it starts by calculating its needs using the helper `_needed`, and ends the negotiation if it needs no more sales/supplies
```python
    my_needs = self._needed()
    if my_needs <= 0:
        return ResponseType.END_NEGOTIATION
```
- If the offered quantity is less than its needs, accept the offer. Otherwise reject the offer.
```python
    return ResponseType.ACCEPT_OFFER if offer[QUANTITY] <= my_needs else ResponseType.REJECT_OFFER
```

Most of the code is in the `best_offer` method which calculates the best offer for a negotiation *given the agreements reached so far*. Let's check it line by line:

- The agent checks its needs and returns `None` ending the negotiation if it needs no more sales/supplies. We also get access to the AMI.
```python
    my_needs = self._needed()
    if my_needs <= 0:
        return None
    ami = self.get_nmi(negotiator_id)
    if not ami:
        return None
```

- It then finds out the `Issue` objects corresponding to the quantity and unit-price for this negotiation and initializes an offer (we have 3 issues)
```python
    quantity_issue = ami.issues[QUANTITY]
    unit_price_issue = ami.issues[UNIT_PRICE]
    offer = [-1] * 3
```
- The time is always the current step. 
```python    
    offer[TIME] = self.awi.current_step
```
- The quantity to offer is simply the needs of the agent without mapped within the range of the quantities in the negotiation agenda (note that this may lead the agent to buy more than its needs). 
```python    
    offer[QUANTITY] = max(
        min(my_needs, quantity_issue.max_value), quantity_issue.min_value
    )
```
- Finally, the unit price is the maximum possible unit price if the agent is selling otherwise it is the minimum possible price. Note that `is_selling()` assumes that the agent will never find itself in a middle layer in a deep negotiation. We will alleviate this issue later. 
```python
    if self._is_selling(ami):
        offer[UNIT_PRICE] = unit_price_issue.max_value
    else:
        offer[UNIT_PRICE] = unit_price_issue.min_value
    return tuple(offer)
```

### A (suposedly) better greedy agent

One problem with our `SimpleAgent` is that it does not take price into account in two ways:

- When asked to `propose`, it *always* proposes an offer with the best price for itself. It **never concedes** on prices. In many cases this will lead to disagreement.
- When asked to `respond` to an offer, *it does not even check the price*. This may lead to bad agreements (i.e. very high buying prices/very low selling prices).

We will try to remedie both of these issues in the following agent:

In [ ]:
class BetterAgent(SimpleAgent):
    """A greedy agent based on OneShotAgent with more sane strategy"""
    
    def __init__(self, *args, concession_exponent=0.2, **kwargs):
        super().__init__(*args, **kwargs)
        self._e = concession_exponent

    def propose(self, negotiator_id: str, state) -> "Outcome":
        offer = super().propose(negotiator_id, state)
        if not offer:
            return None
        offer = list(offer)
        offer[UNIT_PRICE] = self._find_good_price(
            self.get_nmi(negotiator_id), state
        )        
        return tuple(offer)

    def respond(self, negotiator_id, state, source=''):
        offer = state.current_offer
        if offer is None:
            return ResponseType.REJECT_OFFER
        response = super().respond(negotiator_id, state, source)
        if response != ResponseType.ACCEPT_OFFER:
            return response
        ami = self.get_nmi(negotiator_id)
        return (
            response if
            self._is_good_price(ami, state, offer[UNIT_PRICE])
            else ResponseType.REJECT_OFFER
        )
        
    def _is_good_price(self, ami, state, price):
        """Checks if a given price is good enough at this stage"""
        mn, mx = self._price_range(ami)
        th = self._th(state.step, ami.n_steps)        
        # a good price is one better than the threshold
        if self._is_selling(ami):
            return (price - mn) >= th * (mx - mn)
        else:
            return (mx - price) >= th * (mx - mn)
    
    def _find_good_price(self, ami, state):
        """Finds a good-enough price conceding linearly over time"""        
        mn, mx = self._price_range(ami)
        th = self._th(state.step, ami.n_steps)
        # offer a price that is around th of your best possible price
        if self._is_selling(ami):
            return mn + th * (mx - mn)
        else:
            return mx - th * (mx - mn)
    
    def _price_range(self, ami):
        """Finds the minimum and maximum prices"""
        mn = ami.issues[UNIT_PRICE].min_value
        mx = ami.issues[UNIT_PRICE].max_value           
        return mn, mx
    
    def _th(self, step, n_steps):
        """calculates a descending threshold (0 <= th <= 1)"""
        return ((n_steps - step - 1) / (n_steps - 1)) ** self._e    
        
world, ascores, tscores = try_agent(BetterAgent)

Let's see how well did this agent behave:

In [ ]:
print_type_scores(tscores)

It seems that `BetterAgent` is getting a score that is not any better than `SimpleAgent`. Let's dive into the agent and analyze how it works:

The main idea in `BetterAgent` is to treat the *price* issue separately to avoid the two issues presented earlier:

- **Never conceding during proposal** This is solved in the `propose` method by just overriding the price with a `good-enough` price:
  ```python
    offer[UNIT_PRICE] = self._find_good_price(
            self.get_nmi(negotiator_id), state
        )        
  ```
  As an aside, notice that we needed to convert the offer to a list in order to overwrite the price then back into a tuple to send it to the partner.
- **Never checking prices of offers** This is solved in the `respond` method by checking whether or not the price offered is a `good-enough` price:
  ```python
    return (
            response if
            self._is_good_price(ami, state, offer[UNIT_PRICE])
            else ResponseType.REJECT_OFFER
        )
  ```


What we mean by a `good-enough` price is defined in `_is_good_price` and `_find_good_price` methods. Both start by getting the limits of the unit-price in the negotiation agenda and a threshold value ```th```:

```python
mn, mx = self._price_range(ami, state)
th = self._th(mn, mx, state.step, ami.n_steps)
```

The price range is clear enough. For the threshold ```th``` is a value that starts at $1.0$ and goes down toward $0.0$ over the negotiation time under the control of an agent specific parameter ```_e``` called the concession exponent.
Let's see how does this look for different concession exponents:

In [ ]:
x = np.arange(20)
fig = plt.figure()
for e in [0.1, 0.2, 1.0, 5, 10]:
    a = BetterAgent(concession_exponent=e)
    y = [a._th(i, 20) for i in x]
    plt.plot(x, y, label=f"Concession Exponent: {e}")
    plt.xlabel("Step (Of 20)"); plt.ylabel("Threshold $th$")
    plt.legend()

You can see that the smaller the exponent the more *hard-headed* will the agent be. Setting the concession exponent to $0$ will recover the behavior of the `SimpleAgent` in offering but will make it insisting on an unrealistic best price when responding to partner offers (can you see why?) which is definitely a bad idea. Setting it to $\inf$ will recover the behavior of `SimpleAgent` in responding to offers but will make its offers least favorable for itself in terms of price (can you see why?)

Given this threshold function, we can now define `is_good_price` and `_find_good_price`:

- ```_is_good_price``` simply compares the price given to it to the current threshold defined by multiplying ```th``` by the price range```mx - mn```
   - When selling this is achieved by comparing the difference between the price and minimum price to the curren threshold:
   
    ```python
    return (price - mn) >= th * (mx - mn)
    ```
    You can check that this will give the maximum unit price in the first step and gradually goes down to the minimum unit price in the last step (```n_steps - 1```)
   - When buying we go the other way around (starting at minimum price and going up over time to the maximum price):
   
   ```python
   return (mx - price) >= th * (mx - mn)
   ```
   
- ```_find_good_price``` works in the same fashion but rather than checking the goodness of a price, it simply uses the threshold to generate a ```good-enough``` price:

   ```python
   if self._is_selling(ami):
       return mn + th * (mx - mn)
   else:
       return mx - th * (mx - mn)
   ```
    
#### Why did not this approach work

As you may have noticed, `BetterAgent` is not relly better than `SimpleAgent`. why? The main reason is that price does not really matter that much in the settings for SCML 2023 OneShot because the price range is limited to only two consecutive values (e.g. (9, 10)) which increases the relative importance of avoiding penalties by matching demand and supply.

### Thinking about other negotiations

So far, our agent behaved **indepdendently** in each negotiation without considering what is happening in the others (except when one of them completes changing the amount `secured`). A simple way to consider other negotiations is to use the prices offered in them to limit our concessions. The following agent implements this idea
    

In [ ]:
class AdaptiveAgent(BetterAgent):
    """Considers best price offers received when making its decisions"""

    def before_step(self):
        self._best_selling, self._best_buying = 0.0, float("inf")
                     
    def respond(self, negotiator_id, state, source=''):
        """Save the best price received"""
        offer = state.current_offer
        response = super().respond(negotiator_id, state, source)
        ami = self.get_nmi(negotiator_id)
        if self._is_selling(ami):
            self._best_selling = max(offer[UNIT_PRICE], self._best_selling)
        else:
            self._best_buying = min(offer[UNIT_PRICE], self._best_buying)
        return response
    
    def _price_range(self, ami):
        """Limits the price by the best price received"""
        mn, mx = super()._price_range(ami)
        if self._is_selling(ami):
            mn = max(mn, self._best_selling)
        else:
            mx = min(mx, self._best_buying)
        return mn, mx

world, ascores, tscores = try_agent(AdaptiveAgent)

Let's see how well did this agent behave:

In [ ]:
print_type_scores(tscores)

Almost as good as `SimpleAgent`, at least in this simulation (we will make a more rigorous comparison later). One possiblity here is that the agent became too hard-headed again because now whenever it sees a good price on one negotiation, it insists on it for all the rest. This may not be a good idea sometimes as it may lead to more disagreements. In general *the agent must balance getting good prices with matching its input and output quantities*.

### Learning over time

Up until now, our simple agents did not change their behavior over simulation time. Whatever they do in one day depends only on information about what is happening on that day. This may not be optimal. For one, the agent may be able to learn about different partners over the simulation time. The following agent tries to do that in a *simplistic* manner by changing the limits of the price it is willing to accept based on accomulated statistics per agent.

<div class="alert alert-warning">
Note that this agent is not effective in 2024 because the price range is too small that it really does not matter much which price is agreed upon relative to the ability to avoid penalties by matching supply to demand.
</div>


In [ ]:
class LearningAgent(AdaptiveAgent):
    def __init__(
        self,
        *args,
        acc_price_slack=float("inf"),
        step_price_slack=0.0,
        opp_price_slack=0.0,
        opp_acc_price_slack=0.2,
        range_slack = 0.03,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        self._acc_price_slack = acc_price_slack
        self._step_price_slack = step_price_slack
        self._opp_price_slack = opp_price_slack
        self._opp_acc_price_slack = opp_acc_price_slack
        self._range_slack = range_slack

    def init(self):
        """Initialize the quantities and best prices received so far"""
        super().init()
        self._best_acc_selling, self._best_acc_buying = 0.0, float("inf")
        self._best_opp_selling = defaultdict(float)
        self._best_opp_buying = defaultdict(lambda: float("inf"))
        self._best_opp_acc_selling = defaultdict(float)
        self._best_opp_acc_buying = defaultdict(lambda: float("inf"))        

    def step(self):
        """Initialize the quantities and best prices received for next step"""
        super().step()
        self._best_opp_selling = defaultdict(float)
        self._best_opp_buying = defaultdict(lambda: float("inf"))

    def on_negotiation_success(self, contract, mechanism):
        """Record sales/supplies secured"""
        super().on_negotiation_success(contract, mechanism)

        # update my current best price to use for limiting concession in other
        # negotiations
        up = contract.agreement["unit_price"]
        if self._is_selling(mechanism):
            partner = contract.annotation["buyer"]
            self._best_acc_selling = max(up, self._best_acc_selling)
            self._best_opp_acc_selling[partner] = max(up, self._best_opp_acc_selling[partner])
        else:
            partner = contract.annotation["seller"]
            self._best_acc_buying = min(up, self._best_acc_buying)
            self._best_opp_acc_buying[partner] = min(up, self._best_opp_acc_buying[partner])

    def respond(self, negotiator_id, state, source=''):
        offer = state.current_offer
        # find the quantity I still need and end negotiation if I need nothing more
        response = super().respond(negotiator_id, state, source)
        # update my current best price to use for limiting concession in other
        # negotiations
        ami = self.get_nmi(negotiator_id)
        up = offer[UNIT_PRICE]
        if self._is_selling(ami):
            partner = ami.annotation["buyer"]
            self._best_opp_selling[partner] = max(up, self._best_selling)
        else:
            partner = ami.annotation["seller"]
            self._best_opp_buying[partner] = min(up, self._best_buying)
        return response
    
    def _price_range(self, ami):
        """Limits the price by the best price received"""
        mn = ami.issues[UNIT_PRICE].min_value
        mx = ami.issues[UNIT_PRICE].max_value
        if self._is_selling(ami):
            partner = ami.annotation["buyer"]
            mn = min(mx * (1 - self._range_slack), max(
                [mn]
                + [
                    p * (1 - slack)
                    for p, slack in (
                        (self._best_selling, self._step_price_slack),
                        (self._best_acc_selling, self._acc_price_slack),
                        (self._best_opp_selling[partner], self._opp_price_slack),
                        (
                            self._best_opp_acc_selling[partner],
                            self._opp_acc_price_slack,
                        ),
                    )
                ]
            ))
        else:
            partner = ami.annotation["seller"]
            mx = max(mn * (1 + self._range_slack),  min(
                [mx]
                + [
                    p * (1 + slack)
                    for p, slack in (
                        (self._best_buying, self._step_price_slack),
                        (self._best_acc_buying, self._acc_price_slack),
                        (self._best_opp_buying[partner], self._opp_price_slack),
                        (
                            self._best_opp_acc_buying[partner],
                            self._opp_acc_price_slack,
                        ),
                    )
                ]
            ))
        return mn, mx

world, ascores, tscores = try_agent(LearningAgent)

In [ ]:
print_type_scores(tscores)

The main idea of this agent is to limit the minimum price (for selling) and/or maximum price (for buying) based on events not only in this day but up until now. To do that, the agent accomulates best prices received in the same way `AdaptiveAgent` did in `init()` and `step()`. 

There are four simple differences between this agent and the previous one:

1. It allows a controlled `slack` in price constraining. For example, if an agent of this type received a selling price of $10$, it will limit the minimum price considered *good-enough* in other negotiations to $10 \times (1 - s)$ where $s$ is the slack term (e.g. setting $s=0.1$, leads to accepting offers at a price of $9$ as good enough). This may give the agent more flexibility and most importantly, this parameter **can be learned offline using any appropriate ML method**
2. A best price **per partner** is kept in a `defaultdict()` that is initialized in `init` and updated in the same way the *partner independent* best prices are updated
3. An `accumulated` statistic is kept (one for wll agents and one per agent) that is only initialized in `init()` and keeps being adjusted in `on_negotiation_success` without being reset in `step()`
4. It keeps a `range_slack` to avoid setting too high minimum (or too low maximum) price. In effect, if the `range_slack` is $x$ then prices within the best $100x\%$ of the best price are *always considered good enough*

The main logic is in `_price_range()`. As usual, it starts by getting the price range from the `AMI`:

```python
mn = ami.issues[UNIT_PRICE].min_value
mx = ami.issues[UNIT_PRICE].max_value
```

If the agent is a seller, it finds the partner ID:
```python
partner = ami.annotation["buyer"]
```

Now that the partner is known, we can read the best offers received so far (in this day and over all days) for this partner. We can also retrieve the corresponding two partner-independent best prices with the corresponding slacks:

```python
(
    (self._best_selling, self._step_price_slack),
    (self._best_acc_selling, self._acc_price_slack),
    (self._best_opp_selling[partner], self._opp_price_slack),
    (
        self._best_opp_acc_selling[partner],
        self._opp_acc_price_slack,
    ),
)
```

For each price $p$ and slack value $slack$, we find the corrsponding price limit as:
```python
p * (1 - slack)
```

We then set the *good-enough* price limit to be the **maximum** of these four prices and the minimum price of the current negotiation (adjusted by the `range_slack` value). 
Finally we make sure that this is not too high by taking the **minimum** of this value and the maximum price of the current negotiation.


Given that the utility function of the agent is defined in terms of a *complete set of contracts*, it is not trivial to define a utility function for each negotiation independent from the others (which is why this is an inherently concurrent negotiation world). It may be easier then to think of all negotiations in a synchronized manner. This means that the agent keeps collecting offers from its partners and when it has a *complete set*, it responds to all of them. Moreover, to start negotiations in which the agent finds itself the first propsoer, it needs to define a first proposal for each negotiation. Basing your agent on `SyncOneShotAgent` instead of `OneShotAgent` makes this easier. 

### OneShotSyncAgent

One problem that plagued all of our agents so far is that they have to make decisions (`respond`, `propose`) about negotiations **on the spot**. This makes it difficult to consider **all other negotiations** while making decisions. 

Because the utility function is defined for **a complete set of negotiation agreements** and not for any single negotiation by itself, it makes sense to try to make decisions **centrally** by collecting offers from partners then responding to all of them at once. It is possible to do that by utilizing the response type ```ResponseType.WAIT``` supported by NegMAS but this entails a lot of house-keeping. 

To simplify this task, we provide another base class for agents that does all of this house keeping for you exposing a simple interface that **syncrhonizes** all negotiations (as much as allowed by the underlying platform). The main goal of this base agent is to allow the developer to think about *all negotiations together* but it has some important caveats which we will discuss later.
Here is an example of writing the do-nothing agent in this form:

In [ ]:
from negmas import SAOResponse
class MySyncOneShotDoNothing(OneShotSyncAgent):
    """My Agent that does nothing"""
    def counter_all(self, offers, states):
        """Respond to a set of offers given the negotiation state of each."""
        return dict()
    
    def first_proposals(self):
        """Decide a first proposal on every negotiation. 
        Returning None for a negotiation means ending it."""
        return dict()
    
world, ascores, tscores = try_agent(MySyncOneShotDoNothing)

As you can see, in this case, we need to override `counter_all` to counter offers received from *all* the partners and `first_proposals` to decide a first offer for *each* partner. 

Other than these two negotiation related callbacks, the agent receives an `init` call just after it joins the simulatin and a `step` call after each step. The agent is also informed about failure/success of negotiations through the `on_negotiation_success`/`on_negotiation_failure` callbacks. That is all. A one-shot agent needs to only think about what should it do to respond to each of these six callbacks. All of these callbacks except `counter_all` and `first_proposals` are optional.

#### A not so-good SyncAgent

The main advantage of using the `OneShotSyncAgent` is that you do not need to keep track of state variables (like `secured`, `_supplies` and `_sales` used earlier) and you have a common place to make your decisions about **all** negotiations at the same time. Here is a simple greedy agent using this approach.

In [ ]:
class NaiveSyncAgent(OneShotSyncAgent, BetterAgent):
    """A greedy agent based on OneShotSyncAgent"""
    
    def __init__(self, *args, threshold=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        self._threshold = threshold
    
    def before_step(self):
        super().before_step()
        self.ufun.find_limit(True)
        self.ufun.find_limit(False)
        
    def first_proposals(self):
        """Decide a first proposal on every negotiation. 
        Returning None for a negotiation means ending it."""
        return  dict(zip(
                self.negotiators.keys(), 
                (self.best_offer(_) for _ in self.negotiators.keys())
        ))
    
    def counter_all(self, offers, states):
        """Respond to a set of offers given the negotiation state of each."""
        
        # Initialize all responses by my best options
        responses = {
            k: SAOResponse(ResponseType.REJECT_OFFER, v) 
            for k, v in self.first_proposals().items()
        }
        
        # find how much quantity do I still need
        my_needs = self._needed()
        
        # Am I a seller?
        is_selling = ( self._is_selling(self.get_nmi(_)) for _ in offers.keys() )
        
        # sort my offres by price (descendingly/ascendingly for a seller/buyer)
        sorted_offers = sorted(
            zip(offers.values(), is_selling), 
            key=lambda x: (- x[0][UNIT_PRICE]) if x[1] else x[0][UNIT_PRICE]
        )
        
        # greedly choose offers until my needs are satsified
        secured, outputs, chosen = 0, [], dict()
        for i, k in enumerate(offers.keys()):
            offer, is_output = sorted_offers[i]
            secured += offer[QUANTITY]
            if secured >= my_needs:
                break
            chosen[k] = offer
            outputs.append(is_output)
        
        # calculate the utility of selected offers
        u = self.ufun.from_offers(tuple(chosen.values()), tuple(outputs))
        
        # if the utility of selected offers is high enough, accept them
        rng = self.ufun.max_utility - self.ufun.min_utility
        threshold = self._threshold * rng + self.ufun.min_utility
        if u >= threshold:
            for k, v in chosen.items():
                responses[k] = SAOResponse(ResponseType.ACCEPT_OFFER, None)                
        return responses
    
world, ascores, tscores = try_agent(NaiveSyncAgent)

This agent shows a case of parameterizing your agent so that it can be tested with different hyper-parameters. You do that by passing whatever parameters you like as keyword arguments to the constctor:

```python
def __init__(self, *args, threshold=0.3, **kwargs):
    super().__init__(*args, **kwargs)
    self._threshold = threshold
```

The one paramter we have is a threshold of utility relative to the maximum possile utility that we are willing to accept.

This agent also shows a case in which we use the built-in utility function implemented by the system (see [Section 2.3 of the game description](http://www.yasserm.com/scml/scml2021oneshot.pdf)). This ufun is accessible as `ufun`. By default the ufun will return the profit in dollars for a given set of negotiation outcomes, offers, agreements, or contracts. Note that the ufun assumes that what it is given *is the complete set of agreements and no others will be added to them later*. This value may be positive or negative (loss). In some cases you need to get the utility value normalized to a range between zero and one. This agent will do that. To do this normalization, we need to know the value of maximum and minimum utilities. You can of course solve the corresponding optimziation problem but we did that for you. All you need is call `find_limit` and pass it a boolean (`True` for calculating the highest possible utility and `False` for calculating the lowest possible utility). To avoid doing this calculation repeatedly, you should store the results in `ufun.best` or `ufun.worst` for highest and lowest utility. After that, you can access the maximum possible utility as `max_utility` and minimum possible utility as `min_utility`. We do that in the `before_step()` method (called at the beginning of every day):

```python
 def before_step(self):
    super().init()
    self.ufun.find_limit(True)
    self.ufun.find_limit(False)
```

After this call, we can access `maximum_utility`, `minimum_utility`, `best`, `worst` members of the ufun. As explained earlier, `best` and `worst` give extra information about the conditions for achieving maximum and minimum utility.

We need to implement two methods: `first_proposals` (to generate a good first proposal for each negotiation) and `counter_all` (for countering a set of offers). We inherit from `SimpleAgent` in order to get access to `best_offer` and `_is_selling` methods (we could have repeated them here again of course. Note that, because of the way inheritence works in python, we must inherit from `OneShotSyncAgent` before `SimpleAgent`.

The first set of proposals in `first_proposals` is simply the `best_offer` for each negotiation which is calculated using this generator expression:
```python
(self.best_offer(_) for _ in self.negotiators.keys())
```


Almost all the code now resides in the `counter_all` method. We will go over it here:

- We start by initializing our response by the best offer for each negotiation using `first_proposals` and calculating our needs using `_needed`
```python
responses = {
    k: SAOResponse(ResponseType.REJECT_OFFER, _) 
    for k, v in self.first_proposals().items()
}
my_needs = self._needed(None)
```

- We then sort the offers so that earlier offers have *better* prices for us. For sell offers, this means descendingly and for buy offers ascendingly.
```python
is_selling = ( self._is_selling(self.get_nmi(_)) for _ in offers.keys() )
sorted_offers = sorted(
    zip(offers.values(), is_selling), 
    key=lambda x: (- x[0][UNIT_PRICE]) if x[1] else x[0][UNIT_PRICE]
)
```
- We *greedily* find a set of offers that satisfy all our needs (or as much as possible from them). 
```python
secured, outputs, chosen = 0, [], dict()
for i, k in enumerate(offers.keys()):
    offer, is_output = sorted_offers[i]
    secured += offer[QUANTITY]
    if secured >= my_needs:
        break
    chosen[k] = offer
    outputs.append(is_output)
```
- Finally, we calculate the utility of accepting these *and only these* offers and accept the chosen offers if they provide 70% of the maximum possible utility. Otherwise, we reject all offers sending the default `best_offer` value back.
```python
u = self.ufun.from_offers(tuple(chosen.values()), tuple(outputs))
rng = self.ufun.max_utility - self.ufun.min_utility
threshold = self._threshold * rng + self.ufun.min_utility
if u >= threshold:
    for k, v in chosen.items():
        responses[k] = SAOResponse(ResponseType.ACCEPT_OFFER, None)
return responses
```

Let's see how did it do:

In [ ]:
print_type_scores(tscores)

This base-class simplifies the job of the agent developer by providing a single function (`counter_all`) in which to handle all offers it receive (most of the time, remember that sometimes you will receive a subset of the offers in the call). In principle the agent can then decide to accept a few of these offers and keep negotiating. 

The problem with this agent is that it defines a **good offer** independently for each negotiation which defeats the purpose of having the chance to decide centrally what to do for all negotiations. That is made even less effective by the fact that in SCML 2023, price does not matter that much. In the following section, we design a very simple alternative that tries to resolve this issue

### A better SyncAgent

We start by defining a simple helper function that distributes a given quantity $q$ over $n$ partners.

In [ ]:
def distribute(q: int, n: int) -> list[int]:
    """Distributes n values over m bins with at least one item per bin assuming q > n"""
    from numpy.random import choice
    from collections import Counter
    if q < n:
        lst = [0] * (n - q) + [1] * q
        random.shuffle(lst)
        return lst
        
    if q == n:
        return [1] * n
    r = Counter(choice(n, q - n))    
    return [r.get(_, 0) + 1 for _ in range(n)]

Here are few examples of how it would distribute $10$ units over $4$ partners

In [ ]:
[distribute(10, 4) for _ in range(5)]

In [ ]:
[distribute(2, 4) for _ in range(5)]

We will also need a helper function to find all subsets of a given set (powerset):

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
class SyncRandomAgent(OneShotSyncAgent):
    """An agent that distributes its needs over its partners randomly."""

    def __init__(self, *args, threshold=0.95, **kwargs):
        super().__init__(*args, **kwargs)
        self._threshold = threshold

    def distribute_needs(self) -> dict[str, int]:
        """Distributes my needs randomly over all my partners"""

        dist = dict()
        for needs, all_partners in [
            (self.awi.needed_supplies, self.awi.my_suppliers),
            (self.awi.needed_sales, self.awi.my_consumers),
        ]:
            # find suppliers and consumers still negotiating with me
            partner_ids = [_ for _ in all_partners if _ in self.negotiators.keys()]
            partners = len(partner_ids)            

            # if I need nothing, end all negotiations
            if needs <= 0:
                dist.update(dict(zip(partner_ids, [0] * partners)))
                continue

            # distribute my needs over my (remaining) partners.
            dist.update(dict(zip(partner_ids, distribute(needs, partners))))
        return dist

    def first_proposals(self):
        # just randomly distribute my needs over my partners (with best price for me).
        s, p = self._step_and_price(best_price=True)
        distribution = self.distribute_needs()    
        d = {k: (q, s, p) if q > 0 else None for k, q in distribution.items()}
        return d

    def counter_all(self, offers, states):
        response = dict()
        # process for sales and supplies independently
        for needs, all_partners, issues in [
            (
                self.awi.needed_supplies,
                self.awi.my_suppliers,
                self.awi.current_input_issues,
            ),
            (
                self.awi.needed_sales,
                self.awi.my_consumers,
                self.awi.current_output_issues,
            ),
        ]:
            # get a random price
            price = issues[UNIT_PRICE].rand()
            # find active partners
            partners = {_ for _ in all_partners if _ in offers.keys()}

            # find the set of partners that gave me the best offer set
            # (i.e. total quantity nearest to my needs)
            plist = list(powerset(partners))
            best_diff, best_indx = float("inf"), -1
            for i, partner_ids in enumerate(plist):
                others = partners.difference(partner_ids)
                offered = sum(offers[p][QUANTITY] for p in partner_ids)
                diff = abs(offered - needs)
                if diff < best_diff:
                    best_diff, best_indx = diff, i
                if diff == 0:
                    break

            # If the best combination of offers is good enough, accept them and end all
            # other negotiations
            if best_diff <= self._threshold:
                partner_ids = plist[best_indx]
                others = list(partners.difference(partner_ids))
                response.update({
                    k: SAOResponse(ResponseType.ACCEPT_OFFER, offers[k]) for k in partner_ids
                } | {k: SAOResponse(ResponseType.END_NEGOTIATION, None) for k in others})
                continue

            # If I still do not have a good enough offer, distribute my current needs
            # randomly over my partners.
            distribution = self.distribute_needs()
            response.update(
                {
                    k: SAOResponse(ResponseType.END_NEGOTIATION, None)
                    if q == 0
                    else SAOResponse(
                        ResponseType.REJECT_OFFER, (q, self.awi.current_step, price)
                    )
                    for k, q in distribution.items()
                }
            )        
        return response

    def _step_and_price(self, best_price=False):
        """Returns current step and a random (or max) price"""
        s = self.awi.current_step
        seller = self.awi.is_first_level
        issues = (
            self.awi.current_output_issues if seller else self.awi.current_input_issues
        )
        pmin = issues[UNIT_PRICE].min_value
        pmax = issues[UNIT_PRICE].max_value
        if best_price:
            return s, pmax if seller else pmin
        return s, random.randint(pmin, pmax)
world, ascores, tscores = try_agent(SyncRandomAgent)    

In [ ]:
print_type_scores(tscores)

In [ ]:
print_type_scores(tscores)

This is the highest score we got so far even though that agent is not that intelligent in its decision making. Let's check it in details:

The main idea is to generate offers that will (assuming all accepted) give us all the quantity we need (to buy/sell). Moreover, we accept a set of offers if the total quantity they provide is within some small margin from the quantity we need.


We have a helper method to calculate the quantity we need at any time which simply subtracts the secured quantity from our total exogenous contract quantities:

```python
def _needs(self):
    if self.awi.is_first_level:
        return self.awi.needed_sales
    return self.awi.needed_supplies
``` 

Another helper function (`_step_and_price`) is defined to return the current step and one of the two prices in the agenda:

```python
def _step_and_price(self, best_price = False):
    """Returns current step and a random (or max) price"""
    s = self.awi.current_step
    seller = self.awi.is_first_level
    issues = (
        self.awi.current_output_issues 
        if seller else 
        self.awi.current_input_issues
    )
    pmin = issues[UNIT_PRICE].min_value
    pmax = issues[UNIT_PRICE].max_value
    if best_price:
        return s, pmax
    return s, random.randint(pmin, pmax)
```

The core computation of the agent is implemented in the `distribute_needs()` method which is responsible of calculating a quantity for each partner (notice that price is completely ignored here).

Firstly, we find our partner and our needs and if we do not need anything, we simply return $0$ for all partners:

```python
  partner_ids = list(self.negotiators.keys())
  partners = len(partner_ids)
  needs = self._needs()
  if needs <= 0:
      return dict(zip(partner_ids, [0] * partners))
```

If our needs cannot be distributed over all partners, we select some of our partners randomly and return $0$ for them (as we will see, this will end the negotiation with these partners):

```python
if needs < partners:
    to_end = random.sample(partner_ids, (partners - needs))
    response = dict(zip(to_end, [0] * len(to_end)))
    partner_ids = [_ for _ in partner_ids if _ not in to_end]
    partners = len(partner_ids)
```

Finally, we simply distribute our needs over the remaining partners uniformly:

```python
response.update(dict(zip(partner_ids, distribute(needs, partners))))
```

Now we can move the main part of the agent which consists of the two abstract method implementations (`first_proposals` and `counter_all`).

#### First set of offers

The first set of proposals from the agent use the best price and will distribute the total quantity needed randomly between all partners:
```python
s, p = self._step_and_price(best_price=True)
distribution = self.distribute_needs()    
```

We then just return the quantity for each partner or `None` to end the negotiation if the quantity was $0$
```python
return dict((k, (q, s, p) if q > 0 else None)  for k, q in distribution.items())
```

#### Countering offers
When we receive some offers (in `counter_all`) we start by finding the subset of them (together) that best satisfy our needs up to a predefined threshold (defaulting to zero)

```python
plist = list(powerset(partners))
best_diff, best_indx = float("inf"), -1
for i, partner_ids in enumerate(plist):
    others = partners.difference(partner_ids)
    offered = sum(offers[p][QUANTITY] for p in partner_ids)
    diff = abs(offered - needs)
    if diff < best_diff:
        best_diff, best_indx = diff, i
    if diff == 0:
        break
```

If the best subset satisfies our needs up to a threshold (set as zero by default), we simply accept all of them ending all other negotiations:

```python
if best_diff <= self._threshold:
    partner_ids = plist[best_indx]
    others = list(partners.difference(partner_ids))
    return {
        k: SAOResponse(ResponseType.ACCEPT_OFFER, None) for k in partner_ids
    } | {k: SAOResponse(ResponseType.END_NEGOTIATION, None) for k in others}
```

*Note that we could slightly improve that by only rejecting the remaining offers and offering whatever we still need to buy/sell to them when the threshold is nonezero and the best subset has a total quantity less than our needs. This may improve our results slightly but will complicate the code so we do not pursue it in this tutorial.*

If the best subset does not satisfy our needs up to the predefined threshold, we simply ignore all offers and generate a new random offer for our partners:

```python
distribution = self.distribute_needs()
return {
    k: SAOResponse(ResponseType.END_NEGOTIATION, None)
    if q == 0
    else SAOResponse(ResponseType.REJECT_OFFER, (q, s, p))
    for k, q in distribution.items()
}
```    

*Note that we simply end the negotiation with some partners (selected randomly) if our needs are less than the number of our partners (see `distribute_needs()`.*

#### Possible Improvements

There are obvious ways to improve this agent:

1. When countering offers, we should take into account the history of negotiation with each partner (in this round and previously) to make a more meaningful distribution of quantities over partners. Currently this is just random. We should also consider the probability that our offers will be accepted when deciding how to distribute the quantity we still need over our partners.
1. Choosing which negotiators to end the negotiation with when we need a small quantity to buy/sell, is currently random. We could try to find a way to only end negotiation with negotiators least likely to provide us with our remaining needs.
1. As indicated earlier, we should not just end the negotiation with all unselected partners when we accept some subset of the offers if the threshold was nonzero and the total quantity we are accepting is not enough to satisfy our needs.
1. We should take the number of rounds remiaining in the negotiation when deciding whether to accept a subset of offers (e.g. have a higher threshold near the end of the negotiation), and when deciding what quantities to distribute over our partners (e.g. offer more than what we need near the end of the negotiation under the assumption that only some of them will be accepted).
1. May be consider prices more when approaching our total needs.


In many cases, it may be possible to secure all of the agent's needs (i.e. supplies or sales) using a **single** contract with one of its partners. In such cases, the agent can think about the negotiations it is engaged in as a **competetive negotiation** not very dissimilar from an auction that also allows it to offer. This can lead to a further simplification, the agent can be designed to get **at most one agreement** from the set of negotiation and end all the rest once this is achieved. This is what the `SingleAgreementOneShotAgent` does.

### Limiting the agent to a single agreement 
You can limit your agent to have no more than one agreement every time step by basing your agent on `SingleAgreementOneShotAgent`.

This controller manages a set of negotiations from which only a single one
-- at most -- is likely to result in an agreement. To guarentee a single agreement, pass ```strict=True```.

The general algorithm for this agent is something like this:

- Receive offers from all partners.
- Find the best offer among them by calling the abstract `best_offer`
  method.
- Check if this best offer is acceptable using the abstract `is_acceptable`
  method.

    - If the best offer is acceptable, accept it and end all other negotiations.
    - If the best offer is still not acceptable, then all offers are rejected
      and with the partner who sent it receiving the result of `best_outcome`
      while the rest of the partners receive the result of `make_outcome`.

- The default behavior of `best_outcome` is to return the outcome with
  maximum utility.
- The default behavior of `make_outcome` is to return the best offer
  received in this round if it is valid for the respective negotiation
  and the result of `best_outcome` otherwise.
  
To use this agent, you need to implement three methods:

- `is_acceptable` decides whether an offer is now acceptable. For this simple agent, we accept an offer if it provides us with a decreasing fraction of the maximum utility attainable. Notice that, because we set `mn` to `min_utility`, the agent may actually accept offers with negative utility!! This can easily be fixed by setting `mn` to zero but will lead to much less trade.
```python
mx, mn = self.ufun.max_utility , self.ufun.min_utility
u = (self.ufun(offer) - mn) / (mx - mn) 
return  u >= (1-state.relative_time)
```
- `best_offer` finds the best offer among a set of offers. Here we simply compare their utility
```python
ufuns = [(self.ufun(_), i) for i, _ in enumerate(offers.values())]
keys = list(offers.keys())
return keys[max(ufuns)[1]]
```
- `is_better` which compares two offers from the same negotiator. We simply compare their utility value:
```python
return self.ufun(a) > self.ufun(b)
```

Here is the full agent

In [ ]:
class SimpleSingleAgreementAgent(OneShotSingleAgreementAgent):
    """A greedy agent based on OneShotSingleAgreementAgent"""
    
    def before_step(self):
        self.ufun.find_limit(True) # finds highest utility
        self.ufun.find_limit(False) # finds lowest utility
    
    def is_acceptable(self, offer, source, state) -> bool:        
        mx, mn = self.ufun.max_utility , self.ufun.min_utility
        u = (self.ufun(offer) - mn) / (mx - mn) 
        return  u >= (1-state.relative_time)

    def best_offer(self, offers):
        ufuns = [(self.ufun(_), i) 
                 for i, _ in enumerate(offers.values())]
        keys = list(offers.keys())
        return keys[max(ufuns)[1]]

    def is_better(self, a, b, negotiator, state):
        return self.ufun(a) > self.ufun(b)

world, ascors, tscores = try_agent(SimpleSingleAgreementAgent)

In [ ]:
print_type_scores(tscores)

### Using Independent Negotiators

The SCML game is at its core a concurrent negotiation problem. That is why the default `OneShotAgent` receives all calls from the simulator and responds to them centerally. Nevertheless, you man want to structure your agent in a different way. Let's assume that you found a way to create *independent utility functions* to be used with each of your partners. Assuming that your ufuns can summarize what is expected to happen in all other negotiations, you can use the `OneShotIndNegotiatorsAgent` to build an agent that delegates all of its control to independent negotiators that use these per-partner utility functions. 

In this case, you only need to implement `generate_ufuns()` which is responsible of creating these **independent per-partner utility functions** every day. You need also to specify the negotiator to be used with each partner. By default the agent will create an `AspirationNegotiator` for each of your partners. You can override this default behavior by passing custom `default_negotiator_type`/`default_negotiator_params` to the agent constructor. If you need even more control, you can override `generate_negotiator()` to create a custom negotiator object per partner every day. 

This is an example agent that uses this approach:

In [ ]:
from scml.oneshot import OneShotIndNegotiatorsAgent

class AssumeOthersFailIndNeg(OneShotIndNegotiatorsAgent):
    def generate_ufuns(self):
        return defaultdict(lambda: self.ufun)
world, ascors, tscores = try_agent(AssumeOthersFailIndNeg)
print_type_scores(tscores)

This agent simply uses its **central** ufun for each negotiator. This is not a very good strategy because each negotiator is now assuming that *all other negotiations will fail*. Can you see why?

Let's make a slightly more meaningful agent. This time, the utility function used per partner will try to find a favorable price but will not care about quantity. This is obviously not a good strategy either because the agent may over-contract. 

In [ ]:
from negmas.preferences import LinearAdditiveUtilityFunction, LinearUtilityFunction
from negmas.preferences.value_fun import AffineFun, IdentityFun

class GreedyIndNeg(OneShotIndNegotiatorsAgent):
    def generate_ufuns(self):
        d = dict()
        # generate ufuns that prefer higher prices when selling
        for partner_id in itertools.chain(self.awi.my_consumers, self.awi.my_suppliers):
            issues = self.awi.current_output_issues
            if self.awi.is_system(partner_id):
                continue
            d[partner_id] = LinearUtilityFunction(
                weights=dict(quantity=1.0, time=0.0, unit_price=0.0),
                issues=issues,
                reserved_value=0.0,
            )
        return d

world, ascors, tscores = try_agent(GreedyIndNeg)

print_type_scores(tscores)

#### Using A Negotiator from the Genius Platform

One advantage of this type of agents is that it allows you to use almost any [Genius](http://ii.tudelft.nl/genius/) including most winners of past ANAC competitions as your negotiator once you could come up with a good utility function per partner (not an easy task at all).

To be able to use these genius negotiators, you need to have some extra constraints on your utility function:

- All ufuns are of the type `LinearAdditiveUtilityFunction` or `LinearUtilityFunction`.
- All ufuns are normalized with a maximum value of 1.0. You can
  use `normalize_ufuns=True` to gruarantee that.
- All ufuns have a finite reserved value and at least one outcome is
  above it. You can guarantee that by using `set_reservation=True`.
- All weights of the utility function must be between
  zero and one and the weights must sum to one.


If all of these conditions are satisfied, you can use a Genius based negotiator by following these steps:

1. Install the negmas-genius bridge by running on the terminal (this will download the bridge):
   ```bash
   negmas genius-setup
   ```
2. Run a world or a tournament as usual.

You need to have `java` installed (15.0 or later). *Please note that we do not officially support any Genius agents for SCML. Moreover, due to the way the bridge is implemented, these agents will tend to be much slower than negotiators developed directly in python and may suffer from more timeouts as a result. It may be a good idea to reimplement whatever strategy you prefer in python to guarantee stable performance*.

If your Genius negotiation did not start, you can explicitly run the negmas-genius bridge using one of the following two commands:

- Start the negmas-genius bridbe by running on the terminal:
   ```bash
   negmas genius
   ```
- In some cases, the aforementioned command may not work correctly (specially on windows), you can replace it with:
   ```bash
   java -jar $HOME/nemgas/files/geniusbridge.jar
   ```

The only change you need to do in your code is to choose an appropriate `GeniusNegotiator`. Note that not all agents in the genius platform will work correctly. Here is an example

In [ ]:
# import a specific agent from the Genius platform
from negmas.genius.gnegotiators import Atlas3

class GeniusIndNeg(GreedyIndNeg):
    def __init__(self, *args, **kwargs):
        kwargs["default_negotiator_type"] = Atlas3
        kwargs["normalize_ufuns"] = True
        super().__init__(*args, **kwargs)

world, ascors, tscores = try_agent(GeniusIndNeg)
print_type_scores(tscores)

### More General Simple Agent (advanced)

<div style="color: red;">This section is not necessary for SCML-OneShot 2023 competition because all worlds in the competition will have exactly 2 processes (3 products). It is here for readers interested in making their agents future-proof.</div>

One issue that the `SimpleAgent` had was that it assumed that it is either in the first level of the production chain or in the last level. To make an agent that works anywhere, we need just minor modifications:

In [ ]:
class DeepSimpleAgent(SimpleAgent):
    """A greedy agent based on OneShotSyncAgent that does something 
    when in the middle of the production chain"""
    
    def before_step(self):
        self._sales = self._supplies = 0
        
    def on_negotiation_success(self, contract, mechanism):
        if contract.annotation["product"] == self.awi.my_input_product:
            self._sales += contract.agreement["quantity"]
        else:
            self._supplies += contract.agreement["quantity"]
        
    def _needed(self, negotiator_id):
        summary = self.awi.exogenous_contract_summary
        secured = ( 
            self._sales 
            if self._is_selling(self.get_nmi(negotiator_id)) 
            else self._supplies
        )
        demand = min(summary[0][0], summary[-1][0]) / (self.awi.n_competitors + 1)     
        return  demand - secured

Firstly, we now keep track of our sales and supplies separately:

```python
def before_step(self):
    self._sales = self._supplies = 0

def on_negotiation_success(self, contract, mechanism):
    if contract.annotation["seller"] == self.id:
        self._sales += contract.agreement["quantity"]
    else:
        self._supplies += contract.agreement["quantity"]
```

To find out whether a contract is for sales or supplies, we simply check that the `seller` in the contract annotation is us. 

We need now one more chanage which is to separate the calculation of our needs for supplies and sales:
```python
def _needed(self, negotiator_id):
    summary = self.awi.exogenous_contract_summary
    secured = ( 
        self._sales 
        if self._is_selling(self.get_nmi(negotiator_id)) 
        else self._supplies
    )
    demand = min(summary[0][0], summary[-1][0]) / (self.awi.n_competitors + 1)     
    return  demand - secured
```

here we start by reading the summary information of exogenous contracts into `summary`. This is a list of two valued tuples giving the **total** quantity and **total** price (in that order) of all current exogenous contracts for all products. We also find the amount we secured (depending on whether this is a buy or a sell negotiation) and the number of competitors (i.e. agents in the same production level as us). We assume that we need to buy (and sell) the same quantity as the minimum of the raw material and final product exogenous contracts divided equally between us and our competitors (we add one to `n_competitors` to count ourselves).

Now, let's see how does this agent behave compared with the previous agent in a deep world simulation


In [ ]:
world, ascores, tscores = try_agents([SyncRandomAgent, DeepSimpleAgent], n_processes=3)

Examining the graph above, you can see that `DeepSimple` agents could get contracts when they were in the middle of the production chain. The `Simple` agent on the other hand cannot do so. 

Agents in the package `scml.oneshot.agents` were designed to work in deep production graphs not only in the first and last layer whenever that is possible.

We can check the results now:

In [ ]:
print_type_scores(tscores)

Even though our new `DeepSimpleAgent` was able to get contracts which in the middle, it seems that it did worse than `BetterAgent` in terms of final profits. This may be just a quirk of this specific configuration. We will leave it to the reader to investigate this issue (if they choose to).

### Comparing all agents

Let's run a tournament comparing all agents we developed in this tutorial (we will ignore the do-nothing agents)

In [ ]:
# may take a long time
_, _, tscores = try_agents(
    [
        SyncRandomAgent,
        SimpleAgent, DeepSimpleAgent,
        BetterAgent, LearningAgent,
        AdaptiveAgent, NaiveSyncAgent, 
        SimpleSingleAgreementAgent, 
        GreedyIndNeg, GeniusIndNeg
    ],
    n_trials=20,
    n_processes=2,
    draw=False
)
print_type_scores(tscores)

The way we just compared these agents is not unbiased because not all agents are allowed to control the same factories in the same simulation envoironment. The best way to compare these agents is to run a tournament between them. You already learned how to do that in the previous tutorial and we will not repeate it here.

*If you are running this notebook, please note that the tournament running methods `anac2023_*` may not work within a notebook environment. You can just move your code to a normal python script and it will run correctly*


You can find all the agents available in the `scml` package for the one-shot game under `scml.oneshot.agents` including the ones developed in this tutorial (with some modifications):

In [ ]:
import scml.oneshot.agents as agents
pprint([ _ for _ in agents.__dir__() if _.endswith("Agent")])